In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import copy
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

# Optuna Visualization Tools
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_slice
from optuna.visualization import plot_param_importances

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# ==============================
# 1. LOAD & CONFIG
# ==============================
CHOSEN_CROP = "rice" 
TARGET_COL = f"Y_{CHOSEN_CROP}"
PARQUET_PATH = "Parquet/XY_v2.parquet"
SEQ_LEN     = 5              # try larger windows; model uses past SEQ_LEN years
BATCH_SIZE  = 32
CLIP_QUANT  = 0.01           # 1%-99% clipping to tame outliers
EPOCHS      = 100
PATIENCE    = 15
LR          = 2e-4

df = pd.read_parquet(PARQUET_PATH)
df = df[(df["year"] >= 1982) & (df["year"] <= 2023)].copy()
df = df.dropna(subset=[TARGET_COL]).copy()
print(f"--> Filtered years (1982-2023) and dropped NaN targets. New Shape: {df.shape}")

LAG_1_FEATURE = f'avg_yield_{CHOSEN_CROP}_1y'


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# ============================================================
# 1. PREPROCESSING + SEQUENCE BUILDING  (WITH VERBOSE PRINTS)
# ============================================================
def preprocess_and_build_loaders(
    df: pd.DataFrame,
    chosen_crop: str,
    target_col: str,
    seq_len: int = 5,
    batch_size: int = 32,
    clip_quant: float = 0.01,
):
    """
    Full preprocessing + sequence-building pipeline for panel time series LSTM.
    Now with verbose prints so you can see what each step is doing.
    """

    print("========== PREPROCESSING START ==========")
    print(f"Chosen crop: {chosen_crop}")
    print(f"Target col:  {target_col}")
    print(f"Seq len:     {seq_len}")
    print(f"Batch size:  {batch_size}")
    print("-----------------------------------------")

    df_model = df.copy()
    print(f"Initial df shape: {df_model.shape}")

    # -------------------------------------------
    # 1) Drop rows with NaN target
    # -------------------------------------------
    n_before = len(df_model)
    df_model = df_model[~df_model[target_col].isna()].copy()
    n_after = len(df_model)
    print(f"[Step 1] Dropped rows with NaN target: {n_before - n_after} rows removed")
    print(f"        Remaining rows: {n_after}")

    # -------------------------------------------
    # 2) Target & feature selection
    # -------------------------------------------
    print("[Step 2] Target & feature selection...")
    # Drop other target columns
    all_targets = [c for c in df_model.columns if c.startswith("Y_")]
    drop_other_targets = [c for c in all_targets if c != target_col]
    df_model = df_model.drop(columns=drop_other_targets)
    print(f"        Target columns found: {all_targets}")
    print(f"        Keeping target: {target_col}")
    print(f"        Dropping other targets: {drop_other_targets}")

    # Drop unrelated avg_yield_* columns
    avg_yield_cols = [c for c in df_model.columns if c.startswith("avg_yield_")]
    chosen_prefix = f"avg_yield_{chosen_crop}_"
    keep_avg_yield = [c for c in avg_yield_cols if c.startswith(chosen_prefix)]
    drop_avg_yield = list(set(avg_yield_cols) - set(keep_avg_yield))
    df_model = df_model.drop(columns=drop_avg_yield)

    print(f"        Total avg_yield_* cols: {len(avg_yield_cols)}")
    print(f"        Keeping {len(keep_avg_yield)} for {chosen_crop}")
    print(f"        Dropping {len(drop_avg_yield)} from other crops")

    # -------------------------------------------
    # 3) Add time features (trend over years)
    # -------------------------------------------
    print("[Step 3] Adding time features year_index and year_norm...")
    base_year = df_model["year"].min()
    df_model["year_index"] = df_model["year"] - base_year
    max_index = df_model["year_index"].max() if df_model["year_index"].max() > 0 else 1
    df_model["year_norm"] = df_model["year_index"] / max_index
    print(f"        Base year: {base_year}")
    print(f"        Max year_index: {max_index}")
    print(f"        year range in df: {df_model['year'].min()}–{df_model['year'].max()}")

    # -------------------------------------------
    # 4) Sort for time consistency
    # -------------------------------------------
    print("[Step 4] Sorting by ['area', 'year']...")
    df_model = df_model.sort_values(["area", "year"]).reset_index(drop=True)
    print(f"        After sort shape: {df_model.shape}")

    # -------------------------------------------
    # 5) Define feature columns
    # -------------------------------------------
    feature_cols = [
        c
        for c in df_model.columns
        if c not in ["area", "year", target_col] and not c.startswith("Y_")
    ]
    print("[Step 5] Defining feature columns...")
    print(f"        Selected {len(feature_cols)} input features for prediction.")
    print(f"        First 10 features: {feature_cols[:10]}")

    # -------------------------------------------
    # 6) Group-wise ffill/bfill for feature columns
    # -------------------------------------------
    print("[Step 6] Group-wise ffill/bfill within each area...")
    na_before = df_model[feature_cols].isna().sum().sum()
    df_model[feature_cols] = (
        df_model.groupby("area", group_keys=False)[feature_cols]
        .apply(lambda g: g.ffill().bfill())
    )
    na_after = df_model[feature_cols].isna().sum().sum()
    print(f"        NaNs before ffill/bfill: {na_before}")
    print(f"        NaNs after  ffill/bfill: {na_after}")

    # -------------------------------------------
    # 7) Time-based masks (on rows; for scaling only)
    # -------------------------------------------
    print("[Step 7] Creating time-based masks (rows)...")
    train_mask_rows = df_model["year"] < 2014
    val_mask_rows   = (df_model["year"] >= 2014) & (df_model["year"] <= 2018)
    test_mask_rows  = df_model["year"] >= 2019

    print(f"        Train rows: {int(train_mask_rows.sum())}")
    print(f"        Val rows:   {int(val_mask_rows.sum())}")
    print(f"        Test rows:  {int(test_mask_rows.sum())}")

    # -------------------------------------------
    # 8) Raw arrays + NaN imputation (train-mean)
    # -------------------------------------------
    print("[Step 8] Building raw arrays and imputing remaining NaNs with train mean...")
    X_raw = df_model[feature_cols].values.astype(np.float32)
    y_raw = df_model[[target_col]].values.astype(np.float32)

    train_X = X_raw[train_mask_rows]
    train_mean = np.nanmean(train_X, axis=0)
    nan_inds = np.where(np.isnan(X_raw))
    n_nan_total = len(nan_inds[0])
    print(f"        Remaining NaNs before mean-impute: {n_nan_total}")
    X_raw[nan_inds] = np.take(train_mean, nan_inds[1])
    print("        NaNs after mean-impute: 0")

    # -------------------------------------------
    # 9) Outlier clipping (train-based quantiles)
    # -------------------------------------------
    if clip_quant is not None and 0.0 < clip_quant < 0.5:
        print(f"[Step 9] Clipping outliers using train quantiles {clip_quant:.2f} and {1-clip_quant:.2f}...")
        train_X_no_nan = X_raw[train_mask_rows]
        q_low = np.quantile(train_X_no_nan, clip_quant, axis=0)
        q_high = np.quantile(train_X_no_nan, 1.0 - clip_quant, axis=0)

        n_clipped_cols = 0
        for j in range(train_X_no_nan.shape[1]):
            if q_low[j] < q_high[j]:
                X_raw[:, j] = np.clip(X_raw[:, j], q_low[j], q_high[j])
                n_clipped_cols += 1
        print(f"        Columns clipped: {n_clipped_cols}/{train_X_no_nan.shape[1]}")
    else:
        print("[Step 9] Skipping clipping (clip_quant is None or invalid).")

    # -------------------------------------------
    # 10) Scaling (fit only on train)
    # -------------------------------------------
    print("[Step 10] Scaling X and y (StandardScaler, train-only fit)...")
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_scaled = X_raw.copy()
    y_scaled = y_raw.copy()

    X_scaled[train_mask_rows] = scaler_X.fit_transform(X_raw[train_mask_rows])
    X_scaled[val_mask_rows]   = scaler_X.transform(X_raw[val_mask_rows])
    X_scaled[test_mask_rows]  = scaler_X.transform(X_raw[test_mask_rows])

    y_scaled[train_mask_rows] = scaler_y.fit_transform(y_raw[train_mask_rows])
    y_scaled[val_mask_rows]   = scaler_y.transform(y_raw[val_mask_rows])
    y_scaled[test_mask_rows]  = scaler_y.transform(y_raw[test_mask_rows])

    print("        Scaling complete.")
    print(f"        y_scaled train mean (should be ~0): {y_scaled[train_mask_rows].mean():.4f}")
    print(f"        y_scaled train std  (should be ~1): {y_scaled[train_mask_rows].std():.4f}")

    # -------------------------------------------
    # 11) Rebuild scaled DataFrame for sequences
    # -------------------------------------------
    print("[Step 11] Rebuilding scaled DataFrame for sequence building...")
    df_scaled = df_model[["area", "year"]].copy()
    df_scaled["target_scaled"] = y_scaled.reshape(-1)

    X_df = pd.DataFrame(X_scaled, columns=feature_cols, index=df_model.index)
    df_final = pd.concat([df_scaled, X_df], axis=1)
    print(f"        df_final shape: {df_final.shape}")
    print("        Sample df_final head():")
    print(df_final.head())

    # -------------------------------------------
    # 12) Sequence building with split by TARGET YEAR
    # -------------------------------------------
    print("[Step 12] Building sequences and splitting by TARGET YEAR...")

    def build_sequences_by_split(
        df_seq: pd.DataFrame,
        feat_cols: list,
        target_col_name: str,
        seq_len_inner: int,
    ):
        X_train_list, y_train_list = [], []
        X_val_list, y_val_list     = [], []
        X_test_list, y_test_list   = [], []

        for area_name, g in df_seq.groupby("area"):
            g = g.sort_values("year")
            feats = g[feat_cols].values.astype(np.float32)
            targs = g[target_col_name].values.astype(np.float32)
            years = g["year"].values

            if len(g) < seq_len_inner:
                continue

            # Sliding window: [t-seq_len+1 ... t] -> target at t
            for i in range(len(g) - seq_len_inner + 1):
                X_seq = feats[i : i + seq_len_inner]
                y_t   = targs[i + seq_len_inner - 1]
                y_yr  = years[i + seq_len_inner - 1]

                if y_yr < 2014:
                    X_train_list.append(X_seq)
                    y_train_list.append(y_t)
                elif 2014 <= y_yr <= 2018:
                    X_val_list.append(X_seq)
                    y_val_list.append(y_t)
                else:  # y_yr >= 2019
                    X_test_list.append(X_seq)
                    y_test_list.append(y_t)

        def to_array(x_list, y_list):
            if len(x_list) == 0:
                return (
                    np.empty((0, seq_len_inner, len(feat_cols)), dtype=np.float32),
                    np.empty((0,), dtype=np.float32),
                )
            X_arr = np.stack(x_list).astype(np.float32)
            y_arr = np.array(y_list, dtype=np.float32)
            return X_arr, y_arr

        X_tr, y_tr = to_array(X_train_list, y_train_list)
        X_v, y_v   = to_array(X_val_list, y_val_list)
        X_te, y_te = to_array(X_test_list, y_test_list)

        return X_tr, y_tr, X_v, y_v, X_te, y_te

    X_train, y_train, X_val, y_val, X_test, y_test = build_sequences_by_split(
        df_final,
        feature_cols,
        "target_scaled",
        seq_len,
    )

    print(f"        Sequence shapes:")
    print(f"           Train: {X_train.shape}, targets: {y_train.shape}")
    print(f"           Val:   {X_val.shape}, targets: {y_val.shape}")
    print(f"           Test:  {X_test.shape}, targets: {y_test.shape}")

    # -------------------------------------------
    # 13) Build PyTorch DataLoaders
    # -------------------------------------------
    print("[Step 13] Building PyTorch DataLoaders...")

    def make_loader(X, y, batch_size_inner: int, shuffle: bool):
        X_tensor = torch.from_numpy(X)  # (N, seq_len, num_features)
        y_tensor = torch.from_numpy(y).unsqueeze(-1)  # (N, 1)
        dataset = TensorDataset(X_tensor, y_tensor)
        return DataLoader(dataset, batch_size=batch_size_inner, shuffle=shuffle)

    train_loader = make_loader(X_train, y_train, batch_size, shuffle=True)
    val_loader   = make_loader(X_val,   y_val,   batch_size, shuffle=False)
    test_loader  = make_loader(X_test,  y_test,  batch_size, shuffle=False)

    print("========== PREPROCESSING DONE ==========")
    print(f"Train sequences: {len(train_loader.dataset)}")
    print(f"Val sequences:   {len(val_loader.dataset)}")
    print(f"Test sequences:  {len(test_loader.dataset)}")
    print("========================================")

    return train_loader, val_loader, test_loader, feature_cols, scaler_y

In [ ]:
def compute_baseline_rmse(df, target_col):
    """
    Compute baseline model: y(t) ≈ y(t-1)
    Using raw (unscaled) yearly panel data.
    """
    df_sorted = df.sort_values(["area", "year"]).copy()
    df_sorted["y_lag1"] = df_sorted.groupby("area")[target_col].shift(1)

    # Only test years
    test_mask = df_sorted["year"] >= 2019
    df_test = df_sorted[test_mask]

    y_true = df_test[target_col]
    y_pred = df_test["y_lag1"]

    # Drop rows where lag is not available (first year per country)
    mask = (~y_true.isna()) & (~y_pred.isna())
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]

    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    r2 = r2_score(y_true_clean, y_pred_clean)

    return rmse, r2, y_true_clean.values, y_pred_clean.values


In [ ]:
# Build LSTM data loaders
train_loader, val_loader, test_loader, feature_cols, scaler_y = preprocess_and_build_loaders(
    df=df,
    chosen_crop=CHOSEN_CROP,
    target_col=TARGET_COL,
    seq_len=SEQ_LEN,
    batch_size=BATCH_SIZE,
    clip_quant=CLIP_QUANT
)

# ---- Compute baseline BEFORE training ----
rmse_baseline, r2_baseline, y_test_clean, y_pred_clean = compute_baseline_rmse(df, TARGET_COL)

print(f"\nBaseline model: RMSE={rmse_baseline:.2f}, R²={r2_baseline:.4f}")


In [ ]:
# ============================================================
# 2. MODEL DEFINITION (same as yours, but shape-consistent)
# ============================================================
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.2, activation="ReLU"):
        super().__init__()
        
        # 1. LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        
        # 2. Output head: Linear -> Activation -> Linear
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            getattr(nn, activation)(),  # e.g. nn.ReLU(), nn.Tanh()
            nn.Linear(hidden_dim, 1),
        )
        
    def forward(self, x):
        # x: (batch, seq_len, features)
        out, _ = self.lstm(x)
        last_step_out = out[:, -1, :]    # (batch, hidden_dim)
        prediction = self.fc(last_step_out)  # (batch, 1)
        return prediction                # keep (batch, 1) to match y shape


# ============================================================
# 3. GET DATA LOADERS FROM DF
#    (df must already exist in your environment)
# ============================================================
train_loader, val_loader, test_loader, feature_cols, scaler_y = preprocess_and_build_loaders(
    df=df,
    chosen_crop=CHOSEN_CROP,
    target_col=TARGET_COL,
    seq_len=SEQ_LEN,
    batch_size=BATCH_SIZE,
    clip_quant=CLIP_QUANT,
)

# ============================================================
# 4. INITIALIZE MODEL, OPTIMIZER, LOSS
# ============================================================
model = LSTMRegressor(
    input_dim=len(feature_cols),
    hidden_dim=64,
    num_layers=2,
    dropout=0.2,
    activation="ReLU",
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

# ============================================================
# 5. TRAINING LOOP (with early stopping + RMSE tracking)
# ============================================================
best_rmse = float("inf")
best_weights = None
counter = 0

train_rmse_history = []  # RMSE in original units
val_rmse_history = []    # RMSE in original units

print(f"Starting training on {len(train_loader)} batches...")

for epoch in range(EPOCHS):
    # ---------- TRAIN ----------
    model.train()
    train_loss = 0.0

    for X_b, y_b in train_loader:
        X_b = X_b.to(device)
        y_b = y_b.to(device)  # (batch, 1)

        optimizer.zero_grad()
        pred = model(X_b)     # (batch, 1)
        loss = criterion(pred, y_b)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss_scaled = train_loss / max(len(train_loader), 1)
    train_rmse_scaled = np.sqrt(avg_train_loss_scaled)
    # Convert scaled RMSE to original units via scaler_y
    train_rmse_orig = train_rmse_scaled * scaler_y.scale_[0]
    train_rmse_history.append(train_rmse_orig)

    # ---------- VALIDATION ----------
    model.eval()
    val_preds = []
    val_trues = []

    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b = X_b.to(device)
            pred = model(X_b).cpu().numpy()      # (batch, 1)
            val_preds.append(pred)
            val_trues.append(y_b.numpy())        # (batch, 1)

    if val_preds:
        vp_scaled = np.concatenate(val_preds, axis=0)      # (N, 1)
        vt_scaled = np.concatenate(val_trues, axis=0)      # (N, 1)

        vp_inv = scaler_y.inverse_transform(vp_scaled)     # (N, 1)
        vt_inv = scaler_y.inverse_transform(vt_scaled)     # (N, 1)

        val_rmse = np.sqrt(mean_squared_error(vt_inv, vp_inv))
        val_rmse_history.append(val_rmse)

        print(f"Epoch {epoch+1:03d}: Train RMSE {train_rmse_orig:.4f} | Val RMSE {val_rmse:.4f}")

        # Early stopping based on Val RMSE
        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_weights = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= PATIENCE:
                print(f"Early stopping triggered at epoch {epoch+1}!")
                break
    else:
        print(f"Epoch {epoch+1:03d}: Train RMSE {train_rmse_orig:.4f} | Val RMSE N/A (no val sequences)")

print("Training finished.")

# ============================================================
# 6. PLOT TRAIN vs VAL RMSE (ORIGINAL UNITS)
# ============================================================
plt.figure(figsize=(10, 6))
plt.plot(train_rmse_history, label="Train RMSE (Original Units)")
plt.plot(val_rmse_history, label="Val RMSE (Original Units)")
plt.xlabel("Epochs")
plt.ylabel(f"RMSE (Yield: {CHOSEN_CROP})")
plt.title("Learning Curve: Train vs Validation Error")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Load best weights (if any)
if best_weights is not None:
    model.load_state_dict(best_weights)

In [ ]:
# ============================================================
# 7. FINAL EVALUATION (TRAIN / VAL / TEST, ORIGINAL UNITS)
# ============================================================
def evaluate_loader(model, loader, scaler_y, split_name=""):
    """
    Evaluate model on a given loader:
    - Inverse-transforms y and predictions.
    - Returns RMSE, MAE, R2, preds, trues.
    """
    model.eval()
    preds_scaled = []
    trues_scaled = []

    with torch.no_grad():
        for X_b, y_b in loader:
            X_b = X_b.to(device)
            pred = model(X_b).cpu().numpy()  # (batch, 1)
            preds_scaled.append(pred)
            trues_scaled.append(y_b.numpy()) # (batch, 1)

    if not preds_scaled:
        print(f"[{split_name}] No sequences.")
        return np.nan, np.nan, np.nan, np.array([]), np.array([])

    preds_scaled = np.concatenate(preds_scaled, axis=0)   # (N, 1)
    trues_scaled = np.concatenate(trues_scaled, axis=0)   # (N, 1)

    preds = scaler_y.inverse_transform(preds_scaled).ravel()
    trues = scaler_y.inverse_transform(trues_scaled).ravel()

    rmse = np.sqrt(mean_squared_error(trues, preds))
    mae = mean_absolute_error(trues, preds)
    r2  = r2_score(trues, preds)

    print(f"[{split_name}] RMSE: {rmse:.2f} | MAE: {mae:.2f} | R²: {r2:.4f}")
    return rmse, mae, r2, preds, trues


print("\n=== Final Evaluation ===")
train_rmse, train_mae, train_r2, _, _ = evaluate_loader(model, train_loader, scaler_y, split_name="Train")
val_rmse,   val_mae,   val_r2,   _, _ = evaluate_loader(model, val_loader,   scaler_y, split_name="Val")
test_rmse,  test_mae,  test_r2,  p_test, t_test = evaluate_loader(model, test_loader,  scaler_y, split_name="Test")

In [ ]:
def preprocess_and_build_loaders(
    df: pd.DataFrame,
    chosen_crop: str,
    target_col: str,
    seq_len: int = 5,
    batch_size: int = 32,
    clip_quant: float = 0.01,
    verbose: bool = False,   # NEW: control printing
):
    """Preprocess panel data + build LSTM sequences (clean Optuna-friendly version)"""

    df_model = df.copy()

    # ----------------------------
    # 1) Drop NaN targets
    # ----------------------------
    df_model = df_model[~df_model[target_col].isna()].copy()

    # ----------------------------
    # 2) Target and Feature Selection
    # ----------------------------
    all_targets = [c for c in df_model.columns if c.startswith("Y_")]
    df_model = df_model.drop(columns=[c for c in all_targets if c != target_col])

    avg_yield_cols = [c for c in df_model.columns if c.startswith("avg_yield_")]
    chosen_prefix = f"avg_yield_{chosen_crop}_"
    keep_avg = [c for c in avg_yield_cols if c.startswith(chosen_prefix)]
    drop_avg = list(set(avg_yield_cols) - set(keep_avg))
    df_model = df_model.drop(columns=drop_avg)

    # ----------------------------
    # 3) Add time features
    # ----------------------------
    base_year = df_model["year"].min()
    df_model["year_index"] = df_model["year"] - base_year
    max_index = max(df_model["year_index"].max(), 1)
    df_model["year_norm"] = df_model["year_index"] / max_index

    # ----------------------------
    # 4) Sort
    # ----------------------------
    df_model = df_model.sort_values(["area", "year"]).reset_index(drop=True)

    # ----------------------------
    # 5) Define feature columns
    # ----------------------------
    feature_cols = [
        c for c in df_model.columns
        if c not in ["area", "year", target_col] and not c.startswith("Y_")
    ]

    # ----------------------------
    # 6) ffill/bfill per area
    # ----------------------------
    df_model[feature_cols] = (
        df_model.groupby("area", group_keys=False)[feature_cols]
        .apply(lambda g: g.ffill().bfill())
    )

    # ----------------------------
    # 7) Time-based masks
    # ----------------------------
    train_mask = df_model["year"] < 2014
    val_mask   = (df_model["year"] >= 2014) & (df_model["year"] <= 2018)
    test_mask  = df_model["year"] >= 2019

    # ----------------------------
    # 8) NaN imputation
    # ----------------------------
    X_raw = df_model[feature_cols].values.astype(np.float32)
    y_raw = df_model[[target_col]].values.astype(np.float32)

    train_X = X_raw[train_mask]
    train_mean = np.nanmean(train_X, axis=0)
    nan_idx = np.where(np.isnan(X_raw))
    X_raw[nan_idx] = np.take(train_mean, nan_idx[1])

    # ----------------------------
    # 9) Outlier clipping
    # ----------------------------
    if clip_quant is not None and 0 < clip_quant < 0.5:
        train_vals = X_raw[train_mask]
        q_low = np.quantile(train_vals, clip_quant, axis=0)
        q_high = np.quantile(train_vals, 1 - clip_quant, axis=0)
        for j in range(train_vals.shape[1]):
            if q_low[j] < q_high[j]:
                X_raw[:, j] = np.clip(X_raw[:, j], q_low[j], q_high[j])

    # ----------------------------
    # 10) Scaling
    # ----------------------------
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_scaled = X_raw.copy()
    y_scaled = y_raw.copy()

    X_scaled[train_mask] = scaler_X.fit_transform(X_raw[train_mask])
    X_scaled[val_mask]   = scaler_X.transform(X_raw[val_mask])
    X_scaled[test_mask]  = scaler_X.transform(X_raw[test_mask])

    y_scaled[train_mask] = scaler_y.fit_transform(y_raw[train_mask])
    y_scaled[val_mask]   = scaler_y.transform(y_raw[val_mask])
    y_scaled[test_mask]  = scaler_y.transform(y_raw[test_mask])

    # ----------------------------
    # 11) Rebuild scaled DF
    # ----------------------------
    df_scaled = df_model[["area", "year"]].copy()
    df_scaled["target_scaled"] = y_scaled.reshape(-1)

    X_df = pd.DataFrame(X_scaled, columns=feature_cols, index=df_model.index)
    df_final = pd.concat([df_scaled, X_df], axis=1)

    # ----------------------------
    # 12) Sequence Building
    # ----------------------------
    def build_sequences(df_seq):
        X_tr, y_tr = [], []
        X_val_l, y_val_l = [], []
        X_te, y_te = [], []

        for _, g in df_seq.groupby("area"):
            g = g.sort_values("year")
            feats = g[feature_cols].values
            targs = g["target_scaled"].values
            years = g["year"].values

            if len(g) < seq_len:
                continue

            for i in range(len(g) - seq_len + 1):
                X_seq = feats[i:i+seq_len]
                y_t = targs[i + seq_len - 1]
                yr = years[i + seq_len - 1]

                if yr < 2014:
                    X_tr.append(X_seq); y_tr.append(y_t)
                elif yr <= 2018:
                    X_val_l.append(X_seq); y_val_l.append(y_t)
                else:
                    X_te.append(X_seq); y_te.append(y_t)

        def to_np(X, y):
            return (
                np.array(X, dtype=np.float32) if len(X) else np.empty((0, seq_len, len(feature_cols))),
                np.array(y, dtype=np.float32)
            )

        return to_np(X_tr, y_tr), to_np(X_val_l, y_val_l), to_np(X_te, y_te)

    (X_train, y_train), (X_val, y_val), (X_test, y_test) = build_sequences(df_final)

    # ----------------------------
    # 13) Loaders
    # ----------------------------
    def make_loader(X, y, shuffle):
        return DataLoader(
            TensorDataset(torch.from_numpy(X), torch.from_numpy(y).unsqueeze(-1)),
            batch_size=batch_size,
            shuffle=shuffle,
        )

    return (
        make_loader(X_train, y_train, True),
        make_loader(X_val, y_val, False),
        make_loader(X_test, y_test, False),
        feature_cols,
        scaler_y,
    )


In [ ]:
# ============================================================
# 8. OPTUNA TUNING FOR LSTM HYPERPARAMETERS
# ============================================================
import optuna

def objective(trial):
    """
    Optuna objective function:
    - Suggests LSTM + training hyperparameters
    - Rebuilds dataloaders with suggested seq_len
    - Trains on train set, evaluates on val set
    - Returns best validation RMSE (original units)
    """

    # ---- Hyperparameters to tune ----
    hidden_dim = trial.suggest_int("hidden_dim", 32, 128, step=32)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    activation = trial.suggest_categorical("activation", ["ReLU", "Tanh"])
    lr = trial.suggest_float("lr", 1e-5, 5e-3, log=True)
    seq_len = trial.suggest_int("seq_len", 3, 7)

    # ---- Build loaders for this seq_len ----
    train_loader_opt, val_loader_opt, test_loader_opt, feature_cols_opt, scaler_y_opt = preprocess_and_build_loaders(
        df=df,
        chosen_crop=CHOSEN_CROP,
        target_col=TARGET_COL,
        seq_len=seq_len,
        batch_size=BATCH_SIZE,
        clip_quant=CLIP_QUANT,
    )

    # If val set is empty for this seq_len, return large loss
    if len(val_loader_opt.dataset) == 0 or len(train_loader_opt.dataset) == 0:
        return float("inf")

    # ---- Build model for this trial ----
    model_opt = LSTMRegressor(
        input_dim=len(feature_cols_opt),
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout,
        activation=activation,
    ).to(device)

    optimizer_opt = torch.optim.Adam(model_opt.parameters(), lr=lr)
    criterion_opt = nn.MSELoss()

    max_epochs = 80     # you can increase later
    patience = 5
    best_val_rmse = float("inf")
    counter = 0

    for epoch in range(max_epochs):
        # ---------- TRAIN ----------
        model_opt.train()
        train_loss = 0.0

        for X_b, y_b in train_loader_opt:
            X_b, y_b = X_b.to(device), y_b.to(device)
            optimizer_opt.zero_grad()
            pred = model_opt(X_b)
            loss = criterion_opt(pred, y_b)
            loss.backward()
            optimizer_opt.step()
            train_loss += loss.item()

        # ---------- VALIDATION ----------
        model_opt.eval()
        val_preds_scaled, val_trues_scaled = [], []
        with torch.no_grad():
            for X_b, y_b in val_loader_opt:
                X_b = X_b.to(device)
                pred = model_opt(X_b).cpu().numpy()
                val_preds_scaled.append(pred)
                val_trues_scaled.append(y_b.numpy())

        if not val_preds_scaled:
            # no val batches
            return float("inf")

        vp_scaled = np.concatenate(val_preds_scaled, axis=0).reshape(-1, 1)
        vt_scaled = np.concatenate(val_trues_scaled, axis=0).reshape(-1, 1)

        # inverse transform to original units
        vp_inv = scaler_y_opt.inverse_transform(vp_scaled)
        vt_inv = scaler_y_opt.inverse_transform(vt_scaled)

        val_rmse = np.sqrt(mean_squared_error(vt_inv, vp_inv))

        # early stopping inside a trial
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

        # report to Optuna (for pruning)
        trial.report(best_val_rmse, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return best_val_rmse


# ============================================================
# 9. RUN OPTUNA STUDY
# ============================================================
study = optuna.create_study(
    direction="minimize",
    study_name="LSTM_optuna",
)
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("=== OPTUNA BEST RESULT ===")
print("Best Val RMSE:", study.best_value)
print("Best Params:", study.best_params)


# ============================================================
# 10. TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS
# ============================================================
best_params = study.best_params

best_hidden_dim = best_params["hidden_dim"]
best_num_layers = best_params["num_layers"]
best_dropout = best_params["dropout"]
best_activation = best_params["activation"]
best_lr = best_params["lr"]
best_seq_len = best_params["seq_len"]

print("\n=== Retraining final LSTM with best Optuna params ===")
print(f"hidden_dim={best_hidden_dim}, num_layers={best_num_layers}, "
      f"dropout={best_dropout}, activation={best_activation}, "
      f"lr={best_lr}, seq_len={best_seq_len}")

# Rebuild loaders with best seq_len
train_loader_opt, val_loader_opt, test_loader_opt, feature_cols_opt, scaler_y_opt = preprocess_and_build_loaders(
    df=df,
    chosen_crop=CHOSEN_CROP,
    target_col=TARGET_COL,
    seq_len=best_seq_len,
    batch_size=BATCH_SIZE,
    clip_quant=CLIP_QUANT,
)

# Define final model
final_model = LSTMRegressor(
    input_dim=len(feature_cols_opt),
    hidden_dim=best_hidden_dim,
    num_layers=best_num_layers,
    dropout=best_dropout,
    activation=best_activation,
).to(device)

optimizer_final = torch.optim.Adam(final_model.parameters(), lr=best_lr)
criterion_final = nn.MSELoss()

best_rmse_final = float("inf")
best_weights_final = None
counter_final = 0

EPOCHS_FINAL = 100
PATIENCE_FINAL = 15

train_rmse_hist_final = []
val_rmse_hist_final = []

for epoch in range(EPOCHS_FINAL):
    # ---------- TRAIN ----------
    final_model.train()
    train_loss = 0.0
    for X_b, y_b in train_loader_opt:
        X_b, y_b = X_b.to(device), y_b.to(device)
        optimizer_final.zero_grad()
        pred = final_model(X_b)
        loss = criterion_final(pred, y_b)
        loss.backward()
        optimizer_final.step()
        train_loss += loss.item()

    avg_train_loss_scaled = train_loss / max(len(train_loader_opt), 1)
    train_rmse_scaled = np.sqrt(avg_train_loss_scaled)
    train_rmse_orig = train_rmse_scaled * scaler_y_opt.scale_[0]
    train_rmse_hist_final.append(train_rmse_orig)

    # ---------- VALIDATION ----------
    final_model.eval()
    val_preds_scaled, val_trues_scaled = [], []
    with torch.no_grad():
        for X_b, y_b in val_loader_opt:
            X_b = X_b.to(device)
            pred = final_model(X_b).cpu().numpy()
            val_preds_scaled.append(pred)
            val_trues_scaled.append(y_b.numpy())

    if val_preds_scaled:
        vp_scaled = np.concatenate(val_preds_scaled, axis=0).reshape(-1, 1)
        vt_scaled = np.concatenate(val_trues_scaled, axis=0).reshape(-1, 1)

        vp_inv = scaler_y_opt.inverse_transform(vp_scaled)
        vt_inv = scaler_y_opt.inverse_transform(vt_scaled)

        val_rmse = np.sqrt(mean_squared_error(vt_inv, vp_inv))
        val_rmse_hist_final.append(val_rmse)

        print(f"[FINAL] Epoch {epoch+1:03d}: Train RMSE {train_rmse_orig:.4f} | Val RMSE {val_rmse:.4f}")

        if val_rmse < best_rmse_final:
            best_rmse_final = val_rmse
            best_weights_final = copy.deepcopy(final_model.state_dict())
            counter_final = 0
        else:
            counter_final += 1
            if counter_final >= PATIENCE_FINAL:
                print(f"[FINAL] Early stopping at epoch {epoch+1}")
                break
    else:
        print(f"[FINAL] Epoch {epoch+1:03d}: Train RMSE {train_rmse_orig:.4f} | Val RMSE N/A")

print("Final training with best Optuna params completed.")

# Load best weights
if best_weights_final is not None:
    final_model.load_state_dict(best_weights_final)

# Plot final learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_rmse_hist_final, label="Train RMSE (Optuna Best)")
plt.plot(val_rmse_hist_final, label="Val RMSE (Optuna Best)")
plt.xlabel("Epochs")
plt.ylabel(f"RMSE (Yield: {CHOSEN_CROP})")
plt.title("Learning Curve: Train vs Validation (Optuna Best)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ============================================================
# 11. EVALUATE FINAL MODEL (TRAIN / VAL / TEST)
# ============================================================
print("\n=== Final Evaluation: Optuna-Tuned LSTM ===")
_ = evaluate_loader(final_model, train_loader_opt, scaler_y_opt, split_name="Train (Optuna)")
_ = evaluate_loader(final_model, val_loader_opt,   scaler_y_opt, split_name="Val (Optuna)")
_ = evaluate_loader(final_model, test_loader_opt,  scaler_y_opt, split_name="Test (Optuna)")


In [ ]:
# Optuna Visualizations
name = f"{CHOSEN_CROP.capitalize()}_Yield_LSTM"

# 1. Optimization History (best value over trials)
fig = plot_optimization_history(study)
fig.update_layout(
    title=f"{name} – Optimization History (Val RMSE)",
    xaxis_title="Trial",
    yaxis_title="Best Val RMSE",
    width=900,
    height=500,
)
fig.show()

# 2. Parallel Coordinate Plot (how hyperparams interact)
fig = plot_parallel_coordinate(study)
fig.update_layout(
    title=f"{name} – Parallel Coordinate Plot",
    width=900,
    height=500,
)
fig.show()

# 3. Slice Plot (effect of each hyperparameter individually)
fig = plot_slice(study)
fig.update_layout(
    title=f"{name} – Slice Plot",
    width=900,
    height=500,
)
fig.show()

# 4. Hyperparameter Importance (which params matter most)
try:
    fig = plot_param_importances(study)
    fig.update_layout(
        title=f"{name} – Hyperparameter Importance",
        width=900,
        height=500,
    )
    fig.show()
except (ValueError, RuntimeError) as e:
    print(f"Could not plot parameter importance: {e}")

In [ ]:
# ============================================================
# EVALUATE INITIAL LSTM & OPTUNA-TUNED LSTM ON TEST SET
# ============================================================

# Initial LSTM (before Optuna)
rmse_lstm_init, mae_lstm_init, r2_lstm_init, y_pred_lstm_init, y_true_lstm_init = evaluate_loader(
    model,                # initial LSTM
    test_loader,
    scaler_y,
    split_name="Test (Initial LSTM)"
)

# Optuna-tuned LSTM (final_model)
rmse_lstm_opt, mae_lstm_opt, r2_lstm_opt, y_pred_lstm_opt, y_true_lstm_opt = evaluate_loader(
    final_model,          # tuned LSTM
    test_loader_opt,
    scaler_y_opt,
    split_name="Test (Optuna LSTM)"
)

# ============================================================
# PERFORMANCE REPORT vs BASELINE
# ============================================================
imp_final_lstm = (rmse_baseline - rmse_lstm_opt) / rmse_baseline * 100

print("\n--- Final LSTM Performance Report (Test Set) ---")
print(f"Baseline Model:   RMSE={rmse_baseline:.2f}, R2={r2_baseline:.4f}")
print(f"Initial LSTM:     RMSE={rmse_lstm_init:.2f}, R2={r2_lstm_init:.4f}")
print(f"Tuned LSTM (Opt): RMSE={rmse_lstm_opt:.2f}, R2={r2_lstm_opt:.4f} "
      f"(RMSE Improved {imp_final_lstm:.2f}%)")

# ============================================================
# PLOTTING: BASELINE vs INITIAL LSTM vs TUNED LSTM
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# Flatten everything
y_base_true  = np.ravel(y_test_clean)
y_base_pred  = np.ravel(y_pred_clean)

y_init_true  = np.ravel(y_true_lstm_init)
y_init_pred  = np.ravel(y_pred_lstm_init)

y_opt_true   = np.ravel(y_true_lstm_opt)
y_opt_pred   = np.ravel(y_pred_lstm_opt)

# Axis limits based on all predictions and truths
all_preds = np.concatenate([y_base_pred, y_init_pred, y_opt_pred])
all_true  = np.concatenate([y_base_true, y_init_true, y_opt_true])

min_val = min(all_preds.min(), all_true.min())
max_val = max(all_preds.max(), all_true.max())

# 1. Baseline Plot
axes[0].scatter(y_base_true, y_base_pred, alpha=0.4, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
axes[0].set_title(f'Baseline Model\nRMSE: {rmse_baseline:.2f} | R²: {r2_baseline:.3f}')
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

# 2. Initial LSTM Plot
axes[1].scatter(y_init_true, y_init_pred, alpha=0.4, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
axes[1].set_title(f'Initial LSTM Model\nRMSE: {rmse_lstm_init:.2f} | R²: {r2_lstm_init:.3f}')
axes[1].set_xlabel("Actual")

# 3. Tuned LSTM Plot
axes[2].scatter(y_opt_true, y_opt_pred, alpha=0.4, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
axes[2].set_title(f'Tuned LSTM Model (Optuna)\nRMSE: {rmse_lstm_opt:.2f} | R²: {r2_lstm_opt:.3f}')
axes[2].set_xlabel("Actual")

plt.suptitle(f'{CHOSEN_CROP.capitalize()} Yield: Baseline vs LSTM Performance (Actual vs Predicted)',
             fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
def build_sequence_years(df, seq_len):
    """
    Reconstruct the target year for each sequence:
    - group by area
    - sort by year
    - slide windows of length seq_len
    - the target year for each sequence = last year in that window
    Split into train/val/test by TRAIN_END_YEAR and VAL_END_YEAR.
    """
    df_model = df.copy()
    df_model = df_model[~df_model[TARGET_COL].isna()].copy()
    df_model = df_model.sort_values(["area", "year"]).reset_index(drop=True)

    seq_years_train = []
    seq_years_val = []
    seq_years_test = []

    for _, g in df_model.groupby("area"):
        g = g.sort_values("year")
        years = g["year"].values

        if len(g) < seq_len:
            continue

        for i in range(len(g) - seq_len + 1):
            yr = years[i + seq_len - 1]
            if yr < TRAIN_END_YEAR:
                seq_years_train.append(yr)
            elif yr < VAL_END_YEAR:
                seq_years_val.append(yr)
            else:
                seq_years_test.append(yr)

    return (
        np.array(seq_years_train, dtype=int),
        np.array(seq_years_val, dtype=int),
        np.array(seq_years_test, dtype=int),
    )


In [ ]:
# ============================================================
# TEST TIMELINE: ACTUAL vs PREDICTED (Optuna LSTM)
# ============================================================

# Get years for all sequences with the tuned sequence length
seq_years_train_opt, seq_years_val_opt, seq_years_test_opt = build_sequence_years(
    df=df,
    seq_len=best_seq_len,
)

# Align test years with test true/pred arrays from evaluate_loader
assert len(seq_years_test_opt) == len(y_true_lstm_opt), \
    f"Mismatch: {len(seq_years_test_opt)} years vs {len(y_true_lstm_opt)} test points"

# Build test-period dataframe
df_test_timeline = pd.DataFrame({
    "Year":      seq_years_test_opt,
    "Actual":    y_true_lstm_opt,
    "Predicted": y_pred_lstm_opt,
})

# Aggregate by year (e.g. 2019–2023)
yearly_test = (
    df_test_timeline
    .groupby("Year")[["Actual", "Predicted"]]
    .mean()
    .sort_index()
)

# ----------------- Plot -----------------
plt.figure(figsize=(12, 6))
ax = plt.gca()

ax.plot(
    yearly_test.index, yearly_test["Actual"],
    marker="o", linewidth=2.5,
    label="Actual Yield (Test)", color="#4B0082"  # purple-ish
)
ax.plot(
    yearly_test.index, yearly_test["Predicted"],
    marker="x", linestyle="--", linewidth=2.0,
    label="Predicted Yield (Optuna LSTM, Test)", color="#FFA500"
)

ax.set_title(f"Test Period Timeline: Actual vs Predicted Yield – {CHOSEN_CROP.capitalize()}",
             fontsize=14)
ax.set_xlabel("Year")
ax.set_ylabel("Yield (hg/ha)")

ax.grid(True, alpha=0.3)
ax.set_xticks(yearly_test.index)
ax.legend(loc="upper left", frameon=True)

plt.tight_layout()
plt.show()
